In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec
import re
import json
import torch
from sklearn.model_selection import train_test_split
import tensorflow as tf
plt.style.use('seaborn')
sns.set_style('whitegrid')
%matplotlib inline

import os

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers

In [ ]:
# 확인 필요
# raw 데이터 임포트
recipe_df = pd.read_csv('/content/drive/Shareddrives/2021 데이터청년캠퍼스/데이터베이스/recipe_db.csv')

In [ ]:
# 학습 준비
recipe = recipe_df
recipe['Vegetarian_class'].unique()

# 'Vegetarian_class'열 값이 결측인 행을 제거
recipe_test = recipe.dropna(subset = ['Vegetarian_class'], axis=0)
recipe_test['Vegetarian_class'].unique()

Data Exploration

In [ ]:
# column간 상관관계 확인
recipe_corr=recipe.corr()
f,ax=plt.subplots(figsize=(10,7))
sns.heatmap(recipe_corr, cmap='viridis')
plt.title("Correlation between features", 
          weight='bold', 
          fontsize=18)
plt.xticks(weight='bold')
plt.yticks(weight='bold')

plt.show() # 대부분의 column은 70%이상이 결측으로, 하얗게 표시됨
# From the correlation heatmap we can not see a strong correlation between features.

In [ ]:
import torch

if torch.cuda.is_available():    

    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [ ]:
# 학습 데이터, 필요 columns 추출
train = pd.DataFrame()
train['menu'] = recipe_test['RCP_NM']
train['recipe'] = recipe_test['RCP_PARTS_DTLS']
train['Vege Class'] = recipe_test['Vegetarian_class']

# 결측치 확인
train.isnull().sum()

# Vege Class 정수로 매핑
train['Vege Class'] = train['Vege Class'].replace(['채소', '달걀', '우유(유제품)', '생선,조개', '가금류', '소,돼지'], [0,1,2,3,4,5])


veg = train['Vege Class'].values.tolist()

In [ ]:
from transformers import BertTokenizer

print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
# 확인 필요
# 식품 분류표
vegetarian_class = pd.read_excel('/content/drive/Shareddrives/2021 데이터청년캠퍼스/데이터베이스/식품별 1g 당 온실가스 배출량(g)_최종.xlsx')

# 식품 분류 
food_num=vegetarian_class['ING_SEQ']
food_name=vegetarian_class['ING_NM']

food_category={100:'소고기',200:'양고기',300:'치즈',400:'젖소',500:'초콜렛',600:'커피',700:'새우',800:'돼지고기',900:'가금류 고기',1000:'양식 해산물',1100:'달걀',1200:'쌀',1300:'두부',1400:'우유,유제품',1500:'토마토',1600:'바닐라빈',1700:'식용유',1800:'올리브유',1900:'카놀라유',2000:'해바라기유',2100:'설탕',2200:'땅콩',2300:'오트밀',2400:'콩류',2500:'밀,호밀',2600:'비트 슈가',2700:'와인',2800:'옥수수',2900:'보리 (맥주)',3000:'베리/포도류',3100:'두유',3200:'카사바',3300:'완두콩',3400:'과일류',3500:'채소류',3600:'배추속채소',3700:'양파/부추속채소',3800:'뿌리채소',3900:'사과,감귤류',4000:'견과류'}
food_list=dict(zip(food_num,food_name))

In [ ]:
# 토큰 max_len 확인
# recipe 재료를 식품 분류표 기준으로 반환

rcp = train['recipe'].values.astype(str).tolist()
max_len=1

for idx,val in enumerate(rcp):
  one=val.split(',')
  if len(one)>max_len:
    max_len=len(one)
  for idx2,val2 in enumerate(one):
    for key,value in food_list.items():
      if value in val2:

        one[idx2]=key//1000
      
  rcp[idx]=one

In [ ]:
# 입력 인자 input_ids
input_ids=[]

for idx,val in enumerate(rcp):
  input_ids_list=[]
  for i in val:
    if str(type(i))!="<class 'int'>":
      input_ids_list.append(0)
    elif str(type(i))=="<class 'int'>":
      input_ids_list.append(i)
  input_ids.append(input_ids_list)

In [ ]:
# 입력 토큰 0으로 패딩
for idx, val in enumerate(input_ids):
  while len(val)!=max_len:
    if len(val)==max_len:
      break
    else:
      val.append(0)

for _ in input_ids:
  print(_)

In [ ]:
# 데이터 증폭 : 입력 토큰을 랜덤으로 섞어 한 데이터당 20개의 데이터로 만듦
import random

rand_list=[]
rand_veg=[]

for idx,val in enumerate(input_ids):
  temp_ids=val.copy()
  temp_veg=veg[idx]
  for i in range(20): #한 input_ids의 줄 당 20번씩 섞기
    random.shuffle(temp_ids)
    one=temp_ids[:]
    rand_list.append(one)
    rand_veg.append(temp_veg)

input_ids.extend(rand_list)
veg.extend(rand_veg)

In [ ]:
# 입력 토큰 attention_masks
"""
attention_masks 만들기
input_ids 존재값은 1 없는 값은 0
"""
attention_masks=[]
for idx,val in enumerate(input_ids):
  attention_masks_list=[]
  for i in val:
    if i!=0:
      attention_masks_list.append(1)
    else:
      attention_masks_list.append(0)
  attention_masks.append(attention_masks_list)

In [ ]:
# 입력 토큰들을 모델 학습을 위해 tensor 형식으로 변경

import torch
input_ids = torch.LongTensor(input_ids)
attention_masks = torch.LongTensor(attention_masks)
labels = torch.tensor(veg

In [ ]:
# 훈련 데이터 셋을 9:1 비율로 train과 validation 데이터셋으로 분류
from torch.utils.data import TensorDataset, random_split

dataset = TensorDataset(input_ids, attention_masks, labels)  #storing the input ids,masks and labels in dataset

train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])  #90-10 train-val split

# 데이터셋 random sampling
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 32

train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset),     #random sampling in training
            batch_size = batch_size 
        )

validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset),    #sequential sampling in validation
            batch_size = batch_size 
        )

# MODEL ; BERT
from transformers import BertForSequenceClassification, AdamW, BertConfig

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 6,   
    output_attentions = False,
    output_hidden_states = False,
)

In [ ]:
# optimizer and scheduler
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, 
                  eps = 1e-8 
                )

from transformers import get_linear_schedule_with_warmup

epochs = 4


total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, 
                                            num_training_steps = total_steps)

In [ ]:
# 정확도 계산 함수
import numpy as np

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# 소요 시간 계산 함수
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    elapsed_rounded = int(round((elapsed)))
    
    return str(datetime.timedelta(seconds=elapsed_rounded))

Training

In [ ]:
# 훈련 loop
import random
import numpy as np


seed_val = 42
epochs=4

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

model.to(device)


training_stats = []

total_t0 = time.time()

for epoch_i in range(0, epochs):
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):
        
        # Printing the progress after every 40 epochs
        if step % 40 == 0 and not step == 0:
            
            elapsed = format_time(time.time() - t0)
            
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        

        result = model(b_input_ids, 
                       token_type_ids=None, 
                       attention_mask=b_input_mask, 
                       labels=b_labels,
                       return_dict=True)

        loss = result.loss
        logits = result.logits

        total_train_loss += loss.item()

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)            

    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in validation_dataloader:

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():        

            result = model(b_input_ids, 
                           token_type_ids=None, 
                           attention_mask=b_input_mask,
                           labels=b_labels,
                           return_dict=True)

        loss = result.loss
        logits = result.logits
            
        total_eval_loss += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        total_eval_accuracy += flat_accuracy(logits, label_ids)
        
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    avg_val_loss = total_eval_loss / len(validation_dataloader)

    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

Prediction

In [ ]:
# 예측 데이터셋
recipe_vali = recipe[recipe['Vegetarian_class'].isnull()]
vege_vali = recipe_vali['Vegetarian_class']

new_recipe = pd.read_csv('/content/drive/Shareddrives/2021 데이터청년캠퍼스/데이터베이스/Add_recipe_db.csv')
new_recipe['Vegetarian_class'] = np.nan
recipe_vali2 = pd.DataFrame()
recipe_vali2 = pd.concat([recipe_vali, new_recipe])

In [ ]:
# pre processing

rcp2 = recipe_vali2['RCP_PARTS_DTLS'].values.astype(str).tolist()

max_len=1

for idx,val in enumerate(rcp2):
  one=val.split(',')
  if len(one)>max_len:
    max_len=len(one)
  for idx2,val2 in enumerate(one):
    for key,value in food_list.items():
      if value in val2:

        one[idx2]=key//1000
      
  rcp2[idx]=one

input_ids=[]

for idx,val in enumerate(rcp2):
  input_ids_list=[]
  for i in val:
    if str(type(i))!="<class 'int'>":
      input_ids_list.append(0)
    elif str(type(i))=="<class 'int'>":
      input_ids_list.append(i)
  input_ids.append(input_ids_list)

for idx, val in enumerate(input_ids):
  while len(val)!=max_len:
    if len(val)==max_len:
      break
    else:
      val.append(0)

attention_masks=[]
for idx,val in enumerate(input_ids):
  attention_masks_list=[]
  for i in val:
    if i!=0:
      attention_masks_list.append(1)
    else:
      attention_masks_list.append(0)
  attention_masks.append(attention_masks_list)

import torch
input_ids = torch.LongTensor(input_ids)
attention_masks = torch.LongTensor(attention_masks)

vali_dataset = TensorDataset(input_ids, attention_masks)
batch_size = 16

vali_dataloader = DataLoader(
    vali_dataset, sampler = SequentialSampler(vali_dataset), batch_size = batch_size
)

In [ ]:
print('Predicting labels for {:,} Vegetarian Class...'.format(len(input_ids)))

model.eval()

predictions = []

for batch in vali_dataloader:
 
  batch = tuple(t.to(device) for t in batch)
  
  b_input_ids, b_input_mask = batch

  with torch.no_grad():

      result = model(b_input_ids, 
                     token_type_ids=None, 
                     attention_mask=b_input_mask,
                     return_dict=True)

  logits = result.logits

  logits = logits.detach().cpu().numpy()
  
  logits=np.argmax(logits,axis=1)
  for i in range(len(logits)):
    predictions.append(logits[i])
    

print('    DONE.')

recipe_vali2['Vegetarian_class'] = predictions

In [ ]:
# vegetarian class 정수 매핑 반환
recipe_vali2['Vegetarian_class'] = recipe_vali2['Vegetarian_class'].replace([0,1,2,3,4,5],['채소', '달걀', '우유(유제품)', '생선,조개', '가금류', '소,돼지'])

In [ ]:
# 예측 완료된 최종 df 제작
recipe_final = pd.concat([recipe_test, recipe_vali2])

recipe_final = recipe_final.reset_index()
recipe_final = recipe_final.drop(['@id'], axis=1)
recipe_final = recipe_final.reset_index()
recipe_final = recipe_final.drop(['index'], axis=1)

In [ ]:
# 예측 완료 df를 csv로 저장
from pathlib import Path
import os

output_file='recipe_final.csv'
output_dir=Path('/content/drive/Shareddrives/2021 데이터청년캠퍼스/데이터베이스')

output_dir.mkdir(parents=True,exist_ok=True)

recipe_final.to_csv(output_dir/output_file)